# этап 1 
## нахождение на видео человека



In [2]:
from ultralytics import YOLO
import cv2
import numpy as np
import supervision as sv
from PIL import Image
import os

In [2]:
!pip install ultralytics
!pip install clearml
!pip install supervision

In [11]:
model = YOLO("yolov8m.pt")

In [4]:
image=cv2.imread("data/push_up1.jpg")  
results = model.predict(image)
result= results[0]
for box in result.boxes:
    class_id = result.names[box.cls[0].item()]
    cords = box.xyxy[0].tolist()
    cords = [round(x) for x in cords]
    conf = round(box.conf[0].item(), 2)
    cv2.rectangle(image, [cords[0],cords[1]], [cords[2],cords[3]], (0, 255, 0), 2)
    print("Object type:", class_id)
    print("Coordinates:", cords)
    print("Probability:", conf)
    print("---")
    cv2.imwrite('output_image.jpg', image)  


0: 640x288 1 person, 1 tv, 175.1ms
Speed: 5.8ms preprocess, 175.1ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 288)
Object type: person
Coordinates: [13, 1046, 1052, 1511]
Probability: 0.75
---
Object type: tv
Coordinates: [485, 416, 968, 721]
Probability: 0.31
---


In [12]:

video = cv2.VideoCapture("data/push_ups2.mp4")
output_path = os.path.join("","output_video_with_contours.avi")
video_info = sv.VideoInfo.from_video_path("data/push_ups2.mp4")
fourcc = cv2.VideoWriter_fourcc(*"XVID")
output_video = cv2.VideoWriter("output_video_with_contours_large.mp4", fourcc, video_info.fps, (video_info.width, video_info.height))
while True:
    ret, frame = video.read()
    if not ret:
        break
    results = model.predict(frame)
    result= results[0]    
    for box in result.boxes:
      class_id = result.names[box.cls[0].item()]
      cords = box.xyxy[0].tolist()
      cords = [round(x) for x in cords]
      conf = round(box.conf[0].item(), 2)
      cv2.rectangle(frame, [cords[0],cords[1]], [cords[2],cords[3]], (0, 255, 0), 2)
      output_video.write(frame)
      # print("Object type:", class_id)
      # print("Coordinates:", cords)
      # print("Probability:", conf)
      print("---")    
        


0: 640x384 1 person, 1 tv, 487.7ms
Speed: 3.2ms preprocess, 487.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)
---
---

0: 640x384 1 person, 1 tv, 470.9ms
Speed: 2.6ms preprocess, 470.9ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)
---
---

0: 640x384 1 person, 1 tv, 1 remote, 452.1ms
Speed: 2.8ms preprocess, 452.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)
---
---
---

0: 640x384 1 person, 1 tv, 451.7ms
Speed: 7.7ms preprocess, 451.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)
---
---

0: 640x384 1 person, 1 bowl, 1 tv, 446.1ms
Speed: 8.0ms preprocess, 446.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)
---
---
---

0: 640x384 1 person, 1 tv, 454.9ms
Speed: 8.0ms preprocess, 454.9ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)
---
---

0: 640x384 1 person, 1 tv, 447.8ms
Speed: 2.5ms preprocess, 447.8ms inference, 1.0ms postprocess per image at shape (1, 3

KeyboardInterrupt: KeyboardInterrupt: 

In [25]:
video = cv2.VideoCapture("data/push_ups1.mp4")
video_info = sv.VideoInfo.from_video_path("data/push_ups1.mp4")
def process_frame(frame: np.ndarray, _) -> np.ndarray:
    results = model(frame, imgsz=1920)[0]
    
    detections = sv.Detections.from_ultralytics(results)

    box_annotator = sv.BoxAnnotator(thickness=4, text_thickness=4, text_scale=2)

    labels = [f"{model.names[class_id]} {confidence:0.2f}" for _, _, confidence, class_id, _ in detections]
    frame = box_annotator.annotate(scene=frame, detections=detections, labels=labels)

    return frame

sv.process_video(source_path="data/push_ups1.mp4", target_path=f"result.mp4", callback=process_frame)


0: 1920x1088 1 person, 4263.8ms
Speed: 18.5ms preprocess, 4263.8ms inference, 2.3ms postprocess per image at shape (1, 3, 1920, 1088)


ValueError: ValueError: too many values to unpack (expected 5)

In [ ]:
video = cv2.VideoCapture("data/push_ups1.mp4")

while True:
    ret, frame = video.read()
    if not ret:
        break

    height, width, channels = frame.shape

    # Подготовка кадра для обнаружения объектов
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Обработка результатов
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                # Отображение прямоугольника и метки класса на кадре
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, classes[class_id], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Отображение кадра
    cv2.imshow("YOLO Video Processing", frame)

    # Выход при нажатии клавиши 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Освобождение ресурсов
video.release()
cv2.destroyAllWindows()